In [ ]:
import json
from bs4 import BeautifulSoup
import pandas as pd

def get_formatted_text(element):
    """Extract and format text by removing any special formatting characters, focusing on the plain text content."""
    if element is None:
        return ''

    text_parts = []
    for part in element.contents:
        if part.name == 'Format':
            # Join the strings and replace newline characters with spaces
            formatted_text = ''.join(part.strings).replace('\n', '')

            # The following checks are now redundant since we're not adding special characters
            # But they indicate where you might handle special formatting logic if needed

            # text_parts.append(formatted_text) can be used directly without conditionals
            text_parts.append(formatted_text)
        else:
            # Replace newline characters with spaces for non-format elements
            text_parts.append(str(part).replace('\n', ''))

    return ''.join(text_parts).strip()

def process_group(group):
    """Extracts details from a single group, including nested formatting."""
    group_details = {
        'Level': group.get('Level'),
        'Count': group.get('Count'),
    }

    for tag in group.find_all(recursive=False):
        # Skip nested groups for simplicity, but you could extend this to handle them similarly
        if tag.name == 'Group':
            continue
        formatted_text = get_formatted_text(tag)
        group_details[tag.name] = formatted_text

    return group_details

def process_headword(headword):
    """Dynamically processes each headword, extracting details for all direct children."""
    # Basic headword information
    headword_data = {
        'ID': headword.get('ID'),
        'SortAs': headword.get('SortAs'),
        'Level': headword.get('Level'),
    }

    # Initialize a container for groups, to be processed separately
    groups_data = []

    # Dynamically process all direct children of the headword
    for child in headword.find_all(recursive=False):
        if child.name == 'Group':
            # Handle groups separately
            groups_data.append(process_group(child))
        else:
            # For non-group elements, apply formatting and add to headword_data
            formatted_text = get_formatted_text(child)
            # Use the tag name as the key and formatted text as the value
            headword_data[child.name] = formatted_text

    # Serialize group details if any groups are present
    if groups_data:
        headword_data['Groups'] = json.dumps(groups_data, ensure_ascii=False)
    else:
        headword_data['Groups'] = json.dumps([])  # Ensure consistency in output

    return headword_data

def parse_xml_to_dataframe(xml_file_path):
    """Parses the XML, returning a DataFrame with headwords and condensed group info."""
    with open(xml_file_path, 'r', encoding='utf-8') as file:
        xml_content = file.read()

    soup = BeautifulSoup(xml_content, 'xml')
    data = [process_headword(hw) for hw in soup.find_all('Headword')]

    return pd.DataFrame(data)

# XML file path
xml_file_path = '/content/Kroatisch-Nederlands.xml'
df_2013 = parse_xml_to_dataframe(xml_file_path)

In [ ]:
df_2013

,ID,SortAs,Level,HW,GR,PR,KG,RZ,P2,LW,...,PL,TX,LS,KN,VI,B,MN,ST,AV,RE
0,0,a1,Incomplete,a1,n/m,,"slovo, glas",,a,d,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,a2,Incomplete,a2,conj,NaN,NaN,,maar,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,a3,Incomplete,a3,m,NaN,NaN,,a,d,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,abeceda,Incomplete,abeced|a,f,NaN,NaN,,alfabet,h,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,abecedni,Incomplete,abecedni,adj,NaN,NaN,,alfabetisch,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36304,36123,žvakaći,Incomplete,žvakać|i,adj,NaN,NaN,,kauw-,h/d,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36305,36124,žvakati,Incomplete,žvakati,"tr, i",NaN,NaN,,kauwen,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36306,36125,žvalav,Incomplete,žvalav,adj,NaN,NaN,,NaN,NaN,...,NaN,met mondhoekkloofjes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36307,36126,žvale,Incomplete,žvale,f,NaN,NaN,,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_2013.Groups[9]

'[{"Level": "2", "Count": "1", "PR": "", "KG": "preuređenje", "P2": "herinrichting", "LW": "d", "RZ": ""}, {"Level": "2", "Count": "2", "PR": "", "KG": "obrad(b)a", "P2": "bewerking", "LW": "d", "RZ": ""}, {"Level": "2", "Count": "3", "PR": "", "KG": "prilagodba", "P2": "aanpassing", "LW": "d"}]'

In [ ]:
df_2013.columns

Index(['ID', 'SortAs', 'Level', 'HW', 'GR', 'PR', 'KG', 'RZ', 'P2', 'LW', 'TR',
       'OC', 'FR', 'PF', 'Groups', 'KZ', 'LT', 'GS', 'UN', 'US', 'LJ', 'PL',
       'TX', 'LS', 'KN', 'VI', 'B', 'MN', 'ST', 'AV', 'RE'],
      dtype='object')

In [ ]:
df_2013.KG.value_counts()

koga               9
što                7
u šahu             6
žena               5
o svjetlu          5
                  ..
glasanje pilića    1
u kartama          1
usnik              1
šištiti            1
površan            1
Name: KG, Length: 1162, dtype: int64

In [ ]:
df_2013.Groups.isna().sum()

0

In [ ]:
df_2013 = df_2013.drop(['ID', 'GR', 'RZ', 'LW', 'SortAs', 'Level', 'TR', 'OC', 'KZ', 'LT', 'LJ', 'AV', 'RE', 'KG', 'PR'], axis=1)

In [ ]:
df_2013[df_2013['Groups']!='[]']

,HW,P2,FR,PF,Groups,GS,UN,US,PL,TX,LS,KN,VI,B,MN,ST
9,adaptacija,NaN,NaN,NaN,"[{""Level"": ""2"", ""Count"": ""1"", ""PR"": """", ""KG"": ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,adaptirati,NaN,NaN,NaN,"[{""Level"": ""1"", ""Count"": ""1"", ""PR"": """", ""GR"": ...",adaptiram,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,adresar,NaN,NaN,NaN,"[{""Level"": ""2"", ""Count"": ""1"", ""PR"": """", ""KG"": ...",adresara,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,afirmira|ti,NaN,NaN,NaN,"[{""Level"": ""1"", ""Count"": ""1"", ""PR"": """", ""GR"": ...",afirmiram,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,agent,NaN,NaN,NaN,"[{""Level"": ""2"", ""Count"": ""1"", ""PR"": """", ""KG"": ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36297,žutiti,NaN,NaN,NaN,"[{""Level"": ""1"", ""Count"": ""1"", ""PR"": """", ""GR"": ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36298,žut|jeti,NaN,NaN,NaN,"[{""Level"": ""1"", ""Count"": ""1"", ""PR"": """", ""GR"": ...",žutim,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36299,žutokljunac,NaN,NaN,NaN,"[{""Level"": ""2"", ""Count"": ""1"", ""PR"": """", ""LS"": ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36307,žvale,NaN,NaN,NaN,"[{""Level"": ""2"", ""Count"": ""1"", ""PR"": """", ""KG"": ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_2013

,HW,P2,FR,PF,Groups,GS,UN,US,PL,TX,LS,KN,VI,B,MN,ST
0,a1,a,od ~ do ž,van a tot z,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,a2,maar,"on radi puno, ~ opet ništa ne zaradi","hij werkt veel, maar toch verdient hij niets",[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a3,a,a-mol,"a-klein, a-mineur",[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,abeced|a,alfabet,po ~i,op alfabetische volgorde,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,abecedni,alfabetisch,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36304,žvakać|i,kauw-,~a guma,kauwgum,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36305,žvakati,kauwen,NaN,NaN,[],žvačem,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36306,žvalav,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,met mondhoekkloofjes,NaN,NaN,NaN,NaN,NaN,NaN
36307,žvale,NaN,NaN,NaN,"[{""Level"": ""2"", ""Count"": ""1"", ""PR"": """", ""KG"": ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Initialize a dictionary to hold the count of '~' for each column
tilda_counts = {}

# Iterate through each column in the DataFrame
for column in df_2013.columns:
    # Use str.contains() to check for '~', sum() to count True values
    # na=False makes str.contains treat NaNs as False
    tilda_counts[column] = df_2013[column].str.contains('~', na=False).sum()

# Display the counts
print(tilda_counts)

{'ID': 0, 'SortAs': 0, 'Level': 0, 'HW': 0, 'GR': 0, 'PR': 0, 'KG': 0, 'RZ': 0, 'P2': 0, 'LW': 0, 'TR': 0, 'OC': 0, 'FR': 9572, 'PF': 0, 'Groups': 3132, 'KZ': 0, 'LT': 0, 'GS': 0, 'UN': 0, 'US': 0, 'LJ': 0, 'PL': 0, 'TX': 0, 'LS': 0, 'KN': 0, 'VI': 0, 'B': 0, 'MN': 0, 'ST': 0, 'AV': 0, 'RE': 0}


In [ ]:
import re

# Function to remove digits from a string in the 'HW' column
def remove_digits(s):
    return re.sub(r'\d+', '', s)

# Remove digits from the 'HW' column
df_2013['HW'] = df_2013['HW'].apply(remove_digits)

# Extract the part of the 'HW' column before the '|'
replacement_text = df_2013['HW'].str.split('|').str[0]

# Function to replace "~" within the same word only if "~" is connected with a letter
def replace_tilda_in_fr_columns(row, replacement_series):
    fr_columns = [col for col in row.index if col.startswith('FR')]

    for column in fr_columns:
        if isinstance(row[column], str):  # Ensure the value is a string
            # Pattern to match "~" followed immediately by a letter, indicating it's part of the word
            pattern_within_word = r'~(?=\w)'
            # Pattern to match "~" that stands alone (surrounded by spaces or at start/end of string)
            pattern_standalone = r'(?:\s|^)~(?:\s|$)'

            # Replacement text from 'HW' up to "|"
            replacement = replacement_series[row.name]
            # Replace "~" within words
            row[column] = re.sub(pattern_within_word, replacement, row[column])
            # Handle standalone "~" by ensuring the replacement also stands alone (surrounded by spaces)
            replacement_for_standalone = f' {replacement} ' if replacement else ' '
            row[column] = re.sub(pattern_standalone, replacement_for_standalone, row[column])

    return row

# Apply the replace function across all rows, targeting only "FR_" columns
df_2013 = df_2013.apply(replace_tilda_in_fr_columns, axis=1, replacement_series=replacement_text)

In [ ]:
df_2013

,ID,SortAs,Level,HW,GR,PR,KG,RZ,P2,LW,...,PL,TX,LS,KN,VI,B,MN,ST,AV,RE
0,0,a1,Incomplete,a,n/m,,"slovo, glas",,a,d,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,a2,Incomplete,a,conj,NaN,NaN,,maar,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,a3,Incomplete,a,m,NaN,NaN,,a,d,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,abeceda,Incomplete,abeced|a,f,NaN,NaN,,alfabet,h,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,abecedni,Incomplete,abecedni,adj,NaN,NaN,,alfabetisch,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36304,36123,žvakaći,Incomplete,žvakać|i,adj,NaN,NaN,,kauw-,h/d,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36305,36124,žvakati,Incomplete,žvakati,"tr, i",NaN,NaN,,kauwen,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36306,36125,žvalav,Incomplete,žvalav,adj,NaN,NaN,,NaN,NaN,...,NaN,met mondhoekkloofjes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36307,36126,žvale,Incomplete,žvale,f,NaN,NaN,,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_2013.columns

Index(['ID', 'SortAs', 'Level', 'HW', 'GR', 'PR', 'KG', 'RZ', 'P2', 'LW', 'TR',
       'OC', 'FR', 'PF', 'Groups', 'KZ', 'LT', 'GS', 'UN', 'US', 'LJ', 'PL',
       'TX', 'LS', 'KN', 'VI', 'B', 'MN', 'ST', 'AV', 'RE'],
      dtype='object')

In [ ]:
df_2013.KG.value_counts()

koga               9
što                7
u šahu             6
žena               5
o svjetlu          5
                  ..
glasanje pilića    1
u kartama          1
usnik              1
šištiti            1
površan            1
Name: KG, Length: 1162, dtype: int64

In [ ]:
df_2013.drop(columns=['ID', 'SortAs', 'Level', 'GR', 'RZ', 'LW', 'TR', 'OC', 'KZ', 'LT', 'PL', 'GS', 'LJ', 'LS', 'MN', 'ST', 'AV',])

,HW,PR,KG,P2,FR,PF,Groups,UN,US,TX,KN,VI,B,RE
0,a,,"slovo, glas",a,od a do ž,van a tot z,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,a,NaN,NaN,maar,"on radi puno, a opet ništa ne zaradi","hij werkt veel, maar toch verdient hij niets",[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a,NaN,NaN,a,a-mol,"a-klein, a-mineur",[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,abeced|a,NaN,NaN,alfabet,po abecedi,op alfabetische volgorde,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,abecedni,NaN,NaN,alfabetisch,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36304,žvakać|i,NaN,NaN,kauw-,žvakaća guma,kauwgum,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
36305,žvakati,NaN,NaN,kauwen,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
36306,žvalav,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,met mondhoekkloofjes,NaN,NaN,NaN,NaN
36307,žvale,NaN,NaN,NaN,NaN,NaN,"[{""Level"": ""2"", ""Count"": ""1"", ""PR"": """", ""KG"": ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# DataFrame with rows where 'Groups' is exactly '[]'
df_with_empty_groups = df_2013[df_2013['Groups'].apply(lambda x: x == '[]')]

# DataFrame with rows where 'Groups' is not exactly '[]'
df_without_empty_groups = df_2013[df_2013['Groups'].apply(lambda x: x != '[]')]

In [ ]:
df_with_empty_groups

,ID,SortAs,Level,HW,GR,PR,KG,RZ,P2,LW,...,PL,TX,LS,KN,VI,B,MN,ST,AV,RE
0,0,a1,Incomplete,a,n/m,,"slovo, glas",,a,d,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,a2,Incomplete,a,conj,NaN,NaN,,maar,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,a3,Incomplete,a,m,NaN,NaN,,a,d,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,abeceda,Incomplete,abeced|a,f,NaN,NaN,,alfabet,h,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,abecedni,Incomplete,abecedni,adj,NaN,NaN,,alfabetisch,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36302,36121,žvačem,Incomplete,žvačem,pres,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,žvakati,NaN,NaN,NaN,NaN,NaN
36303,36122,žvaka,Incomplete,žvaka,f,NaN,NaN,,kauwgum,h/d,...,NaN,NaN,coll,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36304,36123,žvakaći,Incomplete,žvakać|i,adj,NaN,NaN,,kauw-,h/d,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36305,36124,žvakati,Incomplete,žvakati,"tr, i",NaN,NaN,,kauwen,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Parse the JSON strings in the 'Groups' column
df_without_empty_groups['Groups'] = df_without_empty_groups['Groups'].apply(lambda x: json.loads(x))

# Ensure the HW column is included before exploding
df_exploded = df_without_empty_groups.explode('Groups').reset_index().rename(columns={'index': 'original_index'})

# Normalize the exploded data
df_normalized = pd.json_normalize(df_exploded['Groups'])

# Merge back with the original index and HW column
df_final = pd.concat([df_exploded[['original_index', 'HW']].reset_index(drop=True), df_normalized.reset_index(drop=True)], axis=1)

# Check for any duplicated column names and append a number if found
for col in df_final.columns:
    if df_final.columns.tolist().count(col) > 1:  # If the column name appears more than once
        for suffix, occurrence in enumerate(df_final.loc[:, col].columns, start=1):
            df_final.columns.values[occurrence] = f"{col}_{suffix}"

df_final.head()

<ipython-input-41-7efb4c7153f7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_without_empty_groups['Groups'] = df_without_empty_groups['Groups'].apply(lambda x: json.loads(x))


,original_index,HW,Level,Count,PR,KG,P2,LW,RZ,GR,...,TR,LS,VI,LJ,MN,ST,GS,AV,RE,PL
0,9,adaptacija,2,1,,preuređenje,herinrichting,d,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9,adaptacija,2,2,,obrad(b)a,bewerking,d,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9,adaptacija,2,3,,prilagodba,aanpassing,d,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10,adaptirati,1,1,,NaN,NaN,NaN,,"tr, i/p",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10,adaptirati,1,2,,NaN,zich aanpassen,NaN,,"refl, i/p",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_final.iloc[3]

original_index            10
HW                adaptirati
Level                      1
Count                      1
PR                          
KG                       NaN
P2                       NaN
LW                       NaN
RZ                          
GR                   tr, i/p
SE                       NaN
OC                       NaN
FR                       NaN
PF                       NaN
KZ                       NaN
LT                       NaN
US                       NaN
TX                       NaN
B                        NaN
UN                       NaN
KN                       NaN
TR                       NaN
LS                       NaN
VI                       NaN
LJ                       NaN
MN                       NaN
ST                       NaN
GS                       NaN
AV                       NaN
RE                       NaN
PL                       NaN
Name: 3, dtype: object

In [ ]:
df_final

,original_index,HW,Level,Count,PR,KG,P2,LW,RZ,GR,...,TR,LS,VI,LJ,MN,ST,GS,AV,RE,PL
0,9,adaptacija,2,1,,preuređenje,herinrichting,d,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9,adaptacija,2,2,,obrad(b)a,bewerking,d,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9,adaptacija,2,3,,prilagodba,aanpassing,d,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10,adaptirati,1,1,,NaN,NaN,NaN,,"tr, i/p",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10,adaptirati,1,2,,NaN,zich aanpassen,NaN,,"refl, i/p",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12537,36307,žvale,2,1,,za konja,bit,h,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12538,36307,žvale,2,2,,NaN,mondhoekkloofjes,d,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12539,36307,žvale,2,3,,usta,mond,d,NaN,NaN,...,NaN,pej,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12540,36308,žvaliti,1,1,,NaN,tongen,NaN,,"refl, i",...,NaN,sl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_final.columns

Index(['original_index', 'HW', 'Level', 'Count', 'PR', 'KG', 'P2', 'LW', 'RZ',
       'GR', 'SE', 'OC', 'FR', 'PF', 'KZ', 'LT', 'US', 'TX', 'B', 'UN', 'KN',
       'TR', 'LS', 'VI', 'LJ', 'MN', 'ST', 'GS', 'AV', 'RE', 'PL'],
      dtype='object')

In [ ]:
df_final.drop(columns=['Level', 'GR', 'RZ', 'LW', 'TR', 'OC', 'KZ', 'LT', 'PL', 'GS', 'LJ', 'LS', 'MN', 'ST', 'AV',])

,original_index,HW,Count,PR,KG,P2,SE,FR,PF,US,TX,B,UN,KN,VI,RE
0,9,adaptacija,1,,preuređenje,herinrichting,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9,adaptacija,2,,obrad(b)a,bewerking,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9,adaptacija,3,,prilagodba,aanpassing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10,adaptirati,1,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10,adaptirati,2,,NaN,zich aanpassen,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12537,36307,žvale,1,,za konja,bit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12538,36307,žvale,2,,NaN,mondhoekkloofjes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12539,36307,žvale,3,,usta,mond,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12540,36308,žvaliti,1,,NaN,tongen,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Extract the part of the 'HW' column before the '|'
replacement_text = df_final['HW'].str.split('|').str[0]

# Function to replace "~" within the same word only if "~" is connected with a letter
def replace_tilda_in_fr_columns(row, replacement_series):
    fr_columns = [col for col in row.index if col.startswith('FR')]

    for column in fr_columns:
        if isinstance(row[column], str):  # Ensure the value is a string
            # Pattern to match "~" followed immediately by a letter, indicating it's part of the word
            pattern_within_word = r'~(?=\w)'
            # Pattern to match "~" that stands alone (surrounded by spaces or at start/end of string)
            pattern_standalone = r'(?:\s|^)~(?:\s|$)'

            # Replacement text from 'HW' up to "|"
            replacement = replacement_series[row.name]
            # Replace "~" within words
            row[column] = re.sub(pattern_within_word, replacement, row[column])
            # Handle standalone "~" by ensuring the replacement also stands alone (surrounded by spaces)
            replacement_for_standalone = f' {replacement} ' if replacement else ' '
            row[column] = re.sub(pattern_standalone, replacement_for_standalone, row[column])

    return row

# Apply the replace function across all rows, targeting only "FR_" columns
df_final = df_final.apply(replace_tilda_in_fr_columns, axis=1, replacement_series=replacement_text)

In [ ]:
df_final_fr_pf = df_final[['FR', 'PF']]

In [ ]:
df_final_fr_pf.dropna(inplace=True)

<ipython-input-49-700f7302ffce>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_fr_pf.dropna(inplace=True)


In [ ]:
df_final_fr_pf

,FR,PF
8,afirmirao se kao slikar,hij maakte naam als schilder
13,stupiti u akciju,in actie komen
15,spolni akt,geslachtsdaad
55,aranžer izloga,etaleur
64,luksuzni artikl,luxeartikel
...,...,...
12517,operacija žuči,galblaasoperatie
12518,žučna rasprava,een felle discussie
12522,modni žurnal,modeblad
12523,filmski žurnal,filmjournaal


In [ ]:
df_with_empty_groups_fr_pf = df_with_empty_groups[['FR', 'PF']]

In [ ]:
df_with_empty_groups_fr_pf

,FR,PF
0,od a do ž,van a tot z
1,"on radi puno, a opet ništa ne zaradi","hij werkt veel, maar toch verdient hij niets"
2,a-mol,"a-klein, a-mineur"
3,po abecedi,op alfabetische volgorde
4,NaN,NaN
...,...,...
36302,NaN,NaN
36303,NaN,NaN
36304,žvakaća guma,kauwgum
36305,NaN,NaN


In [ ]:
df_with_empty_groups_fr_pf.dropna(inplace=True)

<ipython-input-53-cdebf39fc000>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_empty_groups_fr_pf.dropna(inplace=True)


In [ ]:
df_final

,original_index,HW,Level,Count,PR,KG,P2,LW,RZ,GR,...,TR,LS,VI,LJ,MN,ST,GS,AV,RE,PL
0,9,adaptacija,2,1,,preuređenje,herinrichting,d,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9,adaptacija,2,2,,obrad(b)a,bewerking,d,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9,adaptacija,2,3,,prilagodba,aanpassing,d,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10,adaptirati,1,1,,NaN,NaN,NaN,,"tr, i/p",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10,adaptirati,1,2,,NaN,zich aanpassen,NaN,,"refl, i/p",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12537,36307,žvale,2,1,,za konja,bit,h,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12538,36307,žvale,2,2,,NaN,mondhoekkloofjes,d,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12539,36307,žvale,2,3,,usta,mond,d,NaN,NaN,...,NaN,pej,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12540,36308,žvaliti,1,1,,NaN,tongen,NaN,,"refl, i",...,NaN,sl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Combine the dataframes
combined_df_fr_pf = pd.concat([df_with_empty_groups_fr_pf, df_final_fr_pf], ignore_index=True)

combined_df_fr_pf

,FR,PF
0,od a do ž,van a tot z
1,"on radi puno, a opet ništa ne zaradi","hij werkt veel, maar toch verdient hij niets"
2,a-mol,"a-klein, a-mineur"
3,po abecedi,op alfabetische volgorde
4,ova primjedba ide na tvoju adresu,dit is een opmerking aan jouw adres
...,...,...
14617,operacija žuči,galblaasoperatie
14618,žučna rasprava,een felle discussie
14619,modni žurnal,modeblad
14620,filmski žurnal,filmjournaal


In [ ]:
df_with_empty_groups_hw_p2 = df_with_empty_groups[['HW', 'P2']]

In [ ]:
df_with_empty_groups_hw_p2

,HW,P2
0,a,a
1,a,maar
2,a,a
3,abeced|a,alfabet
4,abecedni,alfabetisch
...,...,...
36302,žvačem,NaN
36303,žvaka,kauwgum
36304,žvakać|i,kauw-
36305,žvakati,kauwen


In [ ]:
df_with_empty_groups_us_p2 = df_with_empty_groups[['US', 'P2']]

In [ ]:
df_with_empty_groups_us_p2.dropna(inplace=True)

<ipython-input-91-72a71af51ec9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_empty_groups_us_p2.dropna(inplace=True)


In [ ]:
df_with_empty_groups_us_p2

,US,P2
24,odvjetnik,advocaat
26,odvjetnički,advocaten-
35,sklonost,affiniteit
73,naglasak,accent
74,naglasiti,benadrukken
...,...,...
36196,živahnost,levensechtheid
36220,žmigati,knipperen
36240,žrtvovati,ontzegging
36260,gorak,bitter


In [ ]:
df_final_us_p2 = df_final[['US', 'P2']]
df_final_us_p2.dropna(inplace=True)
df_final_us_p2

<ipython-input-93-61e3c3595ebb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_us_p2.dropna(inplace=True)


,US,P2
11,poduzetan,actief
12,okretan,vlot
52,apsolvent,
64,proizvod,product
65,članak,artikel
...,...,...
12422,jadikovka 1,geklaag
12425,žalovati,rouwperiode
12461,oženiti I 1,tot vrouw nemen
12482,živahan,levendig


In [ ]:
df_final_hw_p2 = df_final[['HW', 'P2']]
df_final_hw_p2.dropna(inplace=True)
df_final_hw_p2

<ipython-input-94-6784391a2fb1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_hw_p2.dropna(inplace=True)


,HW,P2
0,adaptacija,herinrichting
1,adaptacija,bewerking
2,adaptacija,aanpassing
4,adaptirati,zich aanpassen
5,adresar,adresboekje
...,...,...
12537,žvale,bit
12538,žvale,mondhoekkloofjes
12539,žvale,mond
12540,žvaliti,tongen


In [ ]:
df_final[df_final['KG'].str.contains('za ', na=False)]

,original_index,HW,Level,Count,PR,KG,P2,LW,RZ,GR,...,TR,LS,VI,LJ,MN,ST,GS,AV,RE,PL
35,176,ambulanta,2,2,,bolnička soba za prijam,opnamekamer,d,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,274,aparatić,2,1,,aparatić za zube,beugel,d,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
136,706,bazen,2,1,,bazen za plivanje,zwembad,h,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
267,1085,blok,2,2,,blok za crtanje,tekenblok,h,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
268,1085,blok,2,3,,"blok za kupone, karte itd.",boekje,h,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11656,33922,vrtić,2,1,,vrtić za djecu,kleuterschool,d,,NaN,...,krɛʃ,NaN,NaN,fr,NaN,NaN,NaN,NaN,NaN,NaN
11666,33964,vu|ći,1,1,,vu|ći za sobom,slepen,NaN,,"tr, i",...,NaN,fig,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11796,34296,zagrijan,2,2,,zagrijan za što,enthousiast,NaN,NaN,NaN,...,ɛntu'ʃɑst,coll,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11910,34571,zalistak,2,3,,zalistak za knjigu,schutblad,h,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
mask = df_final['KG'].notna() & df_final['KG'].str.startswith('za ')
df_final.loc[mask, 'KG'] = df_final['HW'] + ' ' + df_final['KG']

In [ ]:
df_final

,original_index,HW,Level,Count,PR,KG,P2,LW,RZ,GR,...,TR,LS,VI,LJ,MN,ST,GS,AV,RE,PL
0,9,adaptacija,2,1,,preuređenje,herinrichting,d,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9,adaptacija,2,2,,obrad(b)a,bewerking,d,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9,adaptacija,2,3,,prilagodba,aanpassing,d,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10,adaptirati,1,1,,NaN,NaN,NaN,,"tr, i/p",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10,adaptirati,1,2,,NaN,zich aanpassen,NaN,,"refl, i/p",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12537,36307,žvale,2,1,,žvale za konja,bit,h,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12538,36307,žvale,2,2,,NaN,mondhoekkloofjes,d,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12539,36307,žvale,2,3,,usta,mond,d,NaN,NaN,...,NaN,pej,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12540,36308,žvaliti,1,1,,NaN,tongen,NaN,,"refl, i",...,NaN,sl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_final_kg_p2 = df_final[['KG', 'P2']]
df_final_kg_p2.dropna(inplace=True)
df_final_kg_p2

<ipython-input-98-6ef43be13cf5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_kg_p2.dropna(inplace=True)


,KG,P2
0,preuređenje,herinrichting
1,obrad(b)a,bewerking
2,prilagodba,aanpassing
5,notes,adresboekje
6,popis,adressenlijst
...,...,...
12530,žuti tisak,sensatiepers
12533,postajati žut,geel worden
12536,ptić,jonge vogel
12537,žvale za konja,bit


In [ ]:
df_final.KG.value_counts()

osoba            61
koga             40
što              27
brojka           19
tramvaj i sl.    17
                 ..
loše slikati      1
žamoriti          1
žuboriti          1
mesno jelo        1
usta              1
Name: KG, Length: 5022, dtype: int64

In [ ]:
df_final.PR.value_counts()

     12469
/        9
Name: PR, dtype: int64

In [ ]:
combined_df_fr_pf

,FR,PF
0,od a do ž,van a tot z
1,"on radi puno, a opet ništa ne zaradi","hij werkt veel, maar toch verdient hij niets"
2,a-mol,"a-klein, a-mineur"
3,po abecedi,op alfabetische volgorde
4,ova primjedba ide na tvoju adresu,dit is een opmerking aan jouw adres
...,...,...
14617,operacija žuči,galblaasoperatie
14618,žučna rasprava,een felle discussie
14619,modni žurnal,modeblad
14620,filmski žurnal,filmjournaal


In [ ]:
combined_df_fr_pf = combined_df_fr_pf.rename(columns={'FR': 'hr', 'PF': 'nl'})

In [ ]:
df_final_kg_p2
df_final_us_p2
df_final_hw_p2

,HW,P2
0,adaptacija,herinrichting
1,adaptacija,bewerking
2,adaptacija,aanpassing
4,adaptirati,zich aanpassen
5,adresar,adresboekje
...,...,...
12537,žvale,bit
12538,žvale,mondhoekkloofjes
12539,žvale,mond
12540,žvaliti,tongen


In [ ]:
df_with_empty_groups_hw_p2

,HW,P2
0,a,a
1,a,maar
2,a,a
3,abeced|a,alfabet
4,abecedni,alfabetisch
...,...,...
36302,žvačem,NaN
36303,žvaka,kauwgum
36304,žvakać|i,kauw-
36305,žvakati,kauwen


In [ ]:
df_empty_hw_p2 = df_with_empty_groups_hw_p2[['HW', 'P2']]
df_final_hw_p2.dropna(inplace=True)
df_final_hw_p2

In [ ]:
df_final_kg_p2

,hr,nl
0,preuređenje,herinrichting
1,obrad(b)a,bewerking
2,prilagodba,aanpassing
5,notes,adresboekje
6,popis,adressenlijst
...,...,...
12530,žuti tisak,sensatiepers
12533,postajati žut,geel worden
12536,ptić,jonge vogel
12537,žvale za konja,bit


In [ ]:
df_with_empty_groups_kg_p2 = df_with_empty_groups[['KG', 'P2']]
df_with_empty_groups_kg_p2.dropna(inplace=True)
df_with_empty_groups_kg_p2

<ipython-input-118-a314866f1f1b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_empty_groups_kg_p2.dropna(inplace=True)


,KG,P2
0,"slovo, glas",a
111,prostakuša,viswijf
174,ugođaj,sfeer
203,po analogiji,naar analogie (van/met)
220,anestetičar,anesthesist
...,...,...
36239,kamen,offersteen
36240,odricanje,ontzegging
36261,mjehur,blaar
36270,zgrada,provinciehuis


In [ ]:
# List of  DataFrames
dataframes = [df_final_kg_p2, df_final_us_p2, df_final_hw_p2, df_with_empty_groups_hw_p2, df_with_empty_groups_us_p2, df_with_empty_groups_kg_p2]

# Dictionary for renaming columns
rename_dict = {'P2': 'nl'}  # Renaming 'p2' to 'nl'

# Loop through each DataFrame and rename the columns
for df in dataframes:
    # First, rename 'p2' to 'nl'
    df.rename(columns=rename_dict, inplace=True)

    # Then, rename other columns to 'hr', if they don't match 'nl'
    df.columns = ['hr' if col not in rename_dict.values() else col for col in df.columns]

# Now, each DataFrame in the list has its columns renamed accordingly

<ipython-input-119-23ec94c338f4>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns=rename_dict, inplace=True)


In [ ]:
combined_df = pd.concat([
    df_final_kg_p2,
    df_final_us_p2,
    df_final_hw_p2,
    combined_df_fr_pf,
    df_with_empty_groups_hw_p2,
    df_with_empty_groups_us_p2,
    df_with_empty_groups_kg_p2
], ignore_index=True)

# If you need to reset the index
combined_df.reset_index(drop=True, inplace=True)

print(combined_df)

                hr             nl
0      preuređenje  herinrichting
1        obrad(b)a      bewerking
2       prilagodba     aanpassing
3            notes    adresboekje
4            popis  adressenlijst
...            ...            ...
65012        kamen     offersteen
65013    odricanje     ontzegging
65014       mjehur          blaar
65015       zgrada  provinciehuis
65016      površan        haastig

[65017 rows x 2 columns]


In [ ]:
import pandas as pd

# Sample setup - replace this with  actual DataFrame combination code
dataframes = [df_final_kg_p2, df_final_us_p2, df_final_hw_p2, combined_df_fr_pf, df_with_empty_groups_hw_p2, df_with_empty_groups_us_p2]
combined_df = pd.concat(dataframes, ignore_index=True)

# Apply condition across both 'hr' and 'nl' columns
# Replace with empty strings if either column in a row has less than 1 character
mask = (combined_df['hr'].str.len() < 1) | (combined_df['nl'].str.len() < 1)
combined_df.loc[mask, ['hr', 'nl']] = ''

# If you need the rows with both columns having less than one character to be dropped instead:
# combined_df = combined_df[~((combined_df['hr'].str.len() < 1) | (combined_df['nl'].str.len() < 1))]

print(combined_df)

                hr              nl
0      preuređenje   herinrichting
1        obrad(b)a       bewerking
2       prilagodba      aanpassing
3            notes     adresboekje
4            popis   adressenlijst
...            ...             ...
63737    živahnost  levensechtheid
63738      žmigati       knipperen
63739    žrtvovati      ontzegging
63740        gorak          bitter
63741        tulum         feestje

[63742 rows x 2 columns]


In [ ]:
combined_df.dropna(inplace=True)

In [ ]:
combined_df

,hr,nl
0,preuređenje,herinrichting
1,obrad(b)a,bewerking
2,prilagodba,aanpassing
3,notes,adresboekje
4,popis,adressenlijst
...,...,...
63737,živahnost,levensechtheid
63738,žmigati,knipperen
63739,žrtvovati,ontzegging
63740,gorak,bitter


In [ ]:
# Regular expression to match the characters () or |
pattern = r'[\(\)\|]'

# Apply the replacement across all string columns
for col in combined_df.select_dtypes(include=['object']):
    combined_df[col] = combined_df[col].str.replace(pattern, '', regex=True)

print(combined_df)

                hr              nl
0      preuređenje   herinrichting
1          obradba       bewerking
2       prilagodba      aanpassing
3            notes     adresboekje
4            popis   adressenlijst
...            ...             ...
63737    živahnost  levensechtheid
63738      žmigati       knipperen
63739    žrtvovati      ontzegging
63740        gorak          bitter
63741        tulum         feestje

[57818 rows x 2 columns]


In [ ]:
combined_df.reset_index(drop=True, inplace=True)

In [ ]:
combined_df

,hr,nl
0,preuređenje,herinrichting
1,obradba,bewerking
2,prilagodba,aanpassing
3,notes,adresboekje
4,popis,adressenlijst
...,...,...
57813,živahnost,levensechtheid
57814,žmigati,knipperen
57815,žrtvovati,ontzegging
57816,gorak,bitter


In [ ]:
df_final_kg_p2 = df_final[[]]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
combined_df

,hr,nl
0,preuređenje,herinrichting
1,obradba,bewerking
2,prilagodba,aanpassing
3,notes,adresboekje
4,popis,adressenlijst
...,...,...
57813,živahnost,levensechtheid
57814,žmigati,knipperen
57815,žrtvovati,ontzegging
57816,gorak,bitter


In [ ]:
combined_df.drop_duplicates(inplace=True)

In [ ]:
combined_df

,hr,nl
0,preuređenje,herinrichting
1,obradba,bewerking
2,prilagodba,aanpassing
3,notes,adresboekje
4,popis,adressenlijst
...,...,...
56081,slikovit,beeldend
56082,živahnost,levensechtheid
56083,žmigati,knipperen
56084,žrtvovati,ontzegging


In [ ]:
combined_df.reset_index(inplace=True, drop=True)

In [ ]:
combined_df

,hr,nl
0,preuređenje,herinrichting
1,obradba,bewerking
2,prilagodba,aanpassing
3,notes,adresboekje
4,popis,adressenlijst
...,...,...
56081,slikovit,beeldend
56082,živahnost,levensechtheid
56083,žmigati,knipperen
56084,žrtvovati,ontzegging


In [ ]:
combined_df.dropna(inplace=True)

In [ ]:
combined_df.to_csv('/content/drive/MyDrive/combined_df.csv')